# 1000 and 500mb Geopotential Heights  - GFS Forecast Maps from Thredds Server via NCSS and Siphon

## Justin Richling
## 11/15/18

https://doi.org/10.6084/m9.figshare.5244637.v1

In [2]:
# Random Library Imports
import subprocess,os,glob,tempfile,re,imageio,webbrowser,io,sys,types,urllib,urllib2,\
time,cStringIO

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Proj 

# Numerical and Scientific Libraries
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import masked_array, units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo
from metpy.constants import g

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# More Image Manipulation Options
from PIL import Image as PILImage

# Ipyhton Options
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display, Image

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap

# URL Manipulation
from urllib import urlopen

# PV widgits imports
import ipywidgets as widgets

# Warnings
import warnings
warnings.filterwarnings('ignore')

## Helper Functions

In [3]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set the Map Projection

In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-130., -70, 20., 60.]

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set a list for the 24 hour forecast time steps

In [ ]:
now = datetime.utcnow()
#now = datetime(2018,12,27,21,0)
today_day = int('{0:%d}'.format(now))
today_year = int('{0:%Y}'.format(now))
today_month = int('{0:%m}'.format(now))
print today_day,today_year,today_month

forecast_times = []
for i in range(4,8):
    forecast_times.append(datetime(today_year,today_month,today_day,i*3,0))
for i in range(0,5):
    forecast_times.append(datetime(today_year,today_month,today_day+1,i*3,0))
forecast_times

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Figure out where the saved maps will go

In [ ]:
# Set a path to save the plots with string format for the date to set the month and day 
im_save_path ="/path/to/saved files/"
print im_save_path

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Start at the top of the thredds catalog

In [5]:
from siphon.catalog import TDSCatalog
top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
for ref in top_cat.catalog_refs:
    print(ref)


Forecast Model Data
Forecast Products and Analyses
Observation Data
Radar Data
Satellite Data
Unidata case studies


## Model Forecast Data

In [6]:
ref = top_cat.catalog_refs['Forecast Model Data']
ref.href

u'http://thredds.ucar.edu/thredds/idd/forecastModels.xml'

In [7]:
new_cat = ref.follow()
list(new_cat.catalog_refs)

['GEFS Members - Analysis',
 'GEFS Members - Forecasts',
 'GEFS Derived Forecast Products',
 'GFS Quarter Degree Analysis',
 'GFS Quarter Degree Forecast',
 'GFS Half Degree Analysis',
 'GFS Half Degree Forecast',
 'GFS One Degree Analysis',
 'GFS One Degree Forecast',
 'GFS Global 1.0 Degree (NOAAPORT)',
 'GFS Pacific 40km',
 'GFS Pacific 20km',
 'GFS Puerto Rico Half Degree',
 'GFS Puerto Rico Quarter Degree',
 'GFS CONUS 80km',
 'GFS CONUS 20km',
 'GFS CONUS 95km',
 'GFS Alaska 20km',
 'GSD HRRR CONUS 3km wrfprs',
 'GSD HRRR CONUS 3km surface',
 'NCEP HRRR CONUS 2.5km Analysis',
 'NCEP HRRR CONUS 2.5km',
 'NAM Alaska 11km',
 'NAM Alaska 45km from NOAAPORT',
 'NAM Alaska 45km from CONDUIT',
 'NAM Alaska 95km',
 'NAM CONUS 12km from NOAAPORT',
 'NAM CONUS 12km from CONDUIT',
 'NAM CONUS 20km',
 'NAM CONUS 40km',
 'NAM CONUS 80km',
 'NAM Polar 90km',
 'NAM Fireweather Nested',
 'Rapid Refresh CONUS 13km',
 'Rapid Refresh CONUS 20km',
 'Rapid Refresh CONUS 40km',
 'SREF CONUS 40km Ensem

## GFS Quarter Degree Data

In [8]:
model = new_cat.catalog_refs[4]
print str(model)
gfs_cat = model.follow()
list(gfs_cat.datasets)

GFS Quarter Degree Forecast


['Full Collection (Reference / Forecast Time) Dataset',
 'Best GFS Quarter Degree Forecast Time Series',
 'Latest Collection for GFS Quarter Degree Forecast']

## Best GFS Quarter Degree Forecast Time Series

In [9]:
ds = gfs_cat.datasets[1]
print "Variable Name:",ds.name
print "Path:",ds.url_path

Variable Name: Best GFS Quarter Degree Forecast Time Series
Path: grib/NCEP/GFS/Global_0p25deg/Best


<h2>----------------------------------------------//---------------------------------------------------------</h2>
<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1><font style="font-size:32px"><center>-- Plotting all of the GFS forecast hours for the current day --</center></font></h1>

<h2><font><center>-- 1000 and 500mb Heights --</center></font></h2>

In [ ]:
for i in forecast_times:
    ds = gfs_cat.datasets[1]
    subset = ds.subset()
    query_data = subset.query()
    query_data.lonlat_box(west=-130, east=-50, south=10, north=60)

# Allow for NetCDF files
    query_data.accept('netcdf4')
    query_data.time(i)
    data = query_data.variables('Geopotential_height_isobaric')

# Finally attempt to access the data
    data = subset.get_data(query_data)
    
    lev_500 = np.where(data.variables['isobaric'][:] == 50000)[0][0]
    hght_500 = data.variables['Geopotential_height_isobaric'][0, lev_500, :, :]
    
    # Create a figure object, title it, and do the plots.
    fig = plt.figure(figsize = (17.,11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax5 = plt.subplot(1,1,1, projection=plotcrs)
    ax5.set_extent(extent, datacrs)

# Add state boundaries to plot
    ax5.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax5.add_feature(country_borders, edgecolor='black', linewidth=1)

# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['Geopotential_height_isobaric'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    
# Plot Title
    plt.title('1000 and 500mb Heights (m)',loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                        # Heights - 1000mb
#---------------------------------------------------------------------------------------------------
    lev_1000 = np.where(data.variables['isobaric'][:] == 100000)[0][0]

    hght_1000 = data.variables['Geopotential_height_isobaric'][0, lev_1000, :, :]
    MIN = hght_1000.min()
    MAX = hght_1000.max()
#print hght_1000.min(),hght_1000.max()
    hght_1000 = ndimage.gaussian_filter(hght_1000, sigma=3, order=0) * units.meter

    clev1000 = np.arange(MIN, MAX, 50)
    cs = ax5.contour(lon, lat, hght_1000.m, clev1000, colors='blue', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

                                        # Heights - 500mb
#---------------------------------------------------------------------------------------------------
    MIN = hght_500.min()
    MAX = hght_500.max()
    #print hght_500.min(),hght_500.max()
    hght_500 = ndimage.gaussian_filter(hght_500, sigma=3, order=0) * units.meter

    clev500 = np.arange(MIN, MAX, 70)
    cs = ax5.contour(lon, lat, hght_500.m, clev500, colors='black', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)


                                        # Vorticity
#---------------------------------------------------------------------------------------------------
#uwnd_500 = data.variables['u-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#vwnd_500 = data.variables['v-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#dx, dy = mpcalc.lat_lon_grid_spacing(lon, lat)#

#lat = data.variables['lat'][:]
#f = mpcalc.coriolis_parameter(np.deg2rad(lat)).to(units('1/sec'))

#avor = mpcalc.vorticity(uwnd_500, vwnd_500, dx, dy, dim_order='yx') + f
#print avor.units
#MAX = avor2.max()
#MIN = avor2.min()
#print MAX*units('dimensionless'),MIN*units('dimensionless')

#avor2 = ndimage.gaussian_filter(avor, sigma=3, order=0) * units('1/s')


#vort_adv = mpcalc.advection(avor2, [uwnd_500, vwnd_500], (dx, dy), dim_order='yx') * 1e9
# Plot Colorfill of Vorticity Advection
#clev_avoradv = np.arange(MIN, MAX, 5)
#cf = ax5.contourf(lon, lat, vort_adv.m, clev_avoradv[clev_avoradv != 0], extend='both',
#                 cmap='bwr', transform=ccrs.PlateCarree())
#cax = plt.subplot(gs[1])
#cb = plt.colorbar(cf, cax=cax, orientation='horizontal', extendrect='True', ticks=clev_avoradv)
#cb.set_label(r'$1/s^2$', size='large')

#---------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------

    
    GFS_1000_500 = im_save_path+"GFS/1000_500mb/"
    if not os.path.isdir(GFS_1000_500):
        os.makedirs(GFS_1000_500)
    fig.savefig(GFS_1000_500+"1000_500mb_Heights_"+file_time+".png",
            bbox_inches='tight',dpi=120)
print 'done.'